In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# Traveling Salesman Problem: Optimize the Optimization

In this notebook we learn how to run problem instances for the famous **Traveling Salesman Problem** (TSP) on Digital Annealer and an emulator. The `Optimizer` class is provided to generate, test and tune an optimization model. We want to understand the problem and get the best solution for our optimization problem. In this exercise you can play with the GUI and see what it does. Use this to get familiar with the possibilities of the ``Optimizer`` framework. The next lesson will introduce, how to program an ``Optimizer`` notebook

## Optimization for the Traveling Salesman Problem
The **Travelling Salesman** is an old style sales representative starting to visit his customers in the morning, traveling from one customer to the next, and who will return back home in the evening. To save travel time and money for gas he tries to bring the visits in an optimal order such that the total length of his round trip becomes a minimum. There are many analogues and extensions of this problem in industries like production and logistics. For more information about TSP or please see [Traveling Salesman Problem on Wikipedia](https://en.wikipedia.org/wiki/Travelling_salesman_problem).

QUBO formulations (Quadratic Unconstraint Binary Optimization) for TSP are well known and besides formulations for many other NP complete problems it can be found in [Lucas, Andrew. (2014). Ising formulations of many NP problems. Frontiers in Physics. 2. 5. 10.3389/fphy.2014.00005.](https://www.frontiersin.org/articles/10.3389/fphy.2014.00005/full). 

For $N$ cities $c=0,1,...N-1$ and $N$ time steps $t=0,1,...N-1$ let the bit $x_{t,c}$ express the presence at that time in the city. The QUBO we have to build consists of two parts to enforce constraints, i.e. being at exactly one place in each time step:
$$
P_{time} = f_{rules} * \sum_{t=0}^{N - 1} \left(1 - \sum_{c=0}^{N - 1} x_{t,c} \right)^2
$$
and visiting each city exactly once:
$$
P_{city} = f_{rules} * \sum_{c=0}^{N - 1} \left(1 - \sum_{t=0}^{N - 1} x_{t,c} \right)^2
$$
As third part we have the minimization target, which is the length of the selected travel route:
$$
P_{distance} = f_{distance} *  \sum_{t=0}^{N - 1} \sum_{c_0=0 \\c_1=0 \\c_0 \ne c_1}^{N - 1} \Delta_{c_0, c_1} x_{t,c_0} x_{(t+1) mod N, c_1} 
$$
All three together give our QUBO:
$$
    HQ = P_{time} + P_{city} + P_{distance}
$$

Recall, that the TSP involves a salesman who has to visit all the cities/stops in the given region, such that all the cities are visited and visited only once, and at the end the salesman returns to the origin city. The objective of the problem is to minimize the total distance traveled by the salesman.

## Implementation
The following cell contains the implementation of the TSP as QUBO. Execute the cell to instantiate the class and create a GUI in the output. You can now go step by step through the tabs. Press the call button an see what happens. When one step is complete you can proceed with the next or you can return anytime to any tab and repeat a previous part of the process. The stations are:

- **Setup scenario**: Load a business model created from your specification into the GUI
- **Build QUBO**: Build a mathematical object (QUBO) that represents the traveling salesmen optimization task
- **Solve Annealing**: Send the QUBO to a solver and get an answer for best found solution
- **Anneal tracker**: Informs about time consumption in the complete process. If you use ``CPU`` as processor and set ``Graphics`` set to ``Single`` or ``All``, then the random walk during annealing is recorded and graphically displayed in this tab. 
- **Report**: Shows in this case a bit map representation of the solution
- **Visualization**: Shows a nice graphical representation of the best found round trip
- **Pareto curve**: Executes and displays a predefined test series modulating some parameters based on the latest manual settings
- **Timeseries**: Executes and displays a predefined test series of growing complexity based on the latest manual settings

Now make yourself familiar with the GUI and the optimization process. Have fun!

In [ ]:
import matplotlib.pyplot as plt

from random import randint 

from dadk.Optimizer import *

from tabulate import tabulate
import networkx as nx

class Optimizer_Demo_TSP_Model(OptimizerModel):
    ##################################################################################################################
    # constructor (defines GUI and test declaration)
    ##################################################################################################################
    def __init__(self, persistent_file="O_01_Optimizer_Demo_TSP.dao"):
        OptimizerModel.__init__(
            self,
            name = 'Demo_UIFunctions',
            load_parameter=[
                {'name':'N', 'type':'int_slider', 'value':12, 'min':2, 'max': 90, 
                 'label':'Size', 'description':'problem size', 'tags':['demo1', 'demo2']},
                {'name':'fixed_city', 'type':'int_slider', 'value':0, 'min':0, 'max':90, 
                 'label':'fixed city', 'description': '', 'tags':['demo1']},
                {'name':'fixed_time', 'type':'int_slider', 'value':0, 'min':0, 'max':90, 
                 'label':'fixed time', 'description':'', 'tags':['demo1']}
            ],            
            build_qubo_parameter=[
                {'name':'factor_rules', 'type':'float_slider', 'value':100.0, 'min':0.0, 'max':10**6,
                 'label':'factor rules'},
                {'name':'factor_distance', 'type':'float_slider', 'value':1.0, 'min':0.0, 'max':10**3,
                 'label':'factor distance'}
            ], 
            #
            # Define default values for the "Solve annealing" tab.
            # The user can modify these fields, or press the button 
            # "Default" and get these values again.
            #
            default_solver_parameter={
                'number_iterations':2000,
                'temperature_end':0.1,
                'offset_increase_rate':5.0},
            #
            # Define calculated values for the "Solve annealing" tab.
            # The user CAN'T modify these fields any more. The value
            # shown here is a default, which is will be overwritten with 
            # the methode self.Optimizer.set_calculated_solver_parameter(...)
            # see end of build_qubo for an example.
            #
            calculated_solver_parameter={'temperature_start': 50000},
            demo_solver_parameter=[{'name':'number_iterations', 'tags':['demo1', 'demo2']},
                                   {'name':'number_runs', 'tags':['demo1']},
                                   'number_replicas'
                                  ],
            # input data for pareto test series
            pareto_setting={
                'cartesian_product': False,
                'var_object':['build_qubo_parameter', 'build_qubo_parameter'],
                'var_attribute':['factor_rules', 'factor_distance'],
                'var_values':[
                    [50, 75, 100, 125, 150],
                    [1.75, 1.50, 1.00, 0.75, 0.50]
                ]
            },
            # input data for performance test series
            timeseries_setting={
                'var_object':['load_parameter'],
                'var_attribute':['N'],
                'var_values':[[ 4, 8, 12, 16, 20, 24]],
                'max_bits': 8192        
            },
            persistent_file=persistent_file
        )

    
    ##################################################################################################################
    # load method (defines content of tab Setup scenario)
    ##################################################################################################################
    def load(self, N=16, fixed_city=0, fixed_time=0, silent=False):
        
        # store paramters in model attributes
        self.N = N
        self.fixed_city = min(fixed_city, N - 1)
        self.fixed_time = min(fixed_time, N - 1)
        
        # calculate distance table
        self.cities = [[100*math.sin(c*2*math.pi/N), 100*math.cos(c*2*math.pi/N)] for c in range(N)]
        
        self.distance_table = [[math.sqrt((self.cities[c_1][0] - self.cities[c_0][0])**2 + (self.cities[c_1][1] - self.cities[c_0][1])**2)
                                 for c_0 in range(N)] for c_1 in range(N)]
        
        # report actions to tab
        self.LOGGER.info(f'Model for circular map with {self.N} cities created.')
        
        if not silent:
            print(f"""
Model for circular map created:
    Number of cities: {self.N:3d}
    Fixed city:       {self.fixed_city:3d}
    Fixed time:       {self.fixed_time:3d}
""")
        # rich output to linked file
        with open("./O_01_distance_table.html", 'w') as detail:
            detail.write('<h1>Distances:</h1>' + tabulate(self.distance_table, range(N), showindex="always", tablefmt="html"))
        self.Optimizer.set_load_details_reference(url = "O_01_distance_table.html")
        
    ##################################################################################################################
    # build_qubo method (defines content of tab Build QUBO)
    ##################################################################################################################
    def build_qubo(self, factor_distance, factor_rules, silent=False):
        
        # variable bit as default are initialized with -1
        constant_bits = np.full((self.N, self.N), -1, np.int8)
        # all bits for fixed_time are set to constant 1
        constant_bits[self.fixed_time, :] = 0
        # all bits for fixed city are set to constant 0
        constant_bits[:, self.fixed_city] = 0
        # bit for fixed_time in fixed_city is set to constant 1
        constant_bits[self.fixed_time, self.fixed_city] = +1                
        # square array x with constant bits is created as VarShapeSet            
        self.qubo_vs = VarShapeSet(BitArrayShape('x', (self.N, self.N), constant_bits=constant_bits, axis_names=['Time', 'City']))        
                        
        #-P_time-------------------------------------------------------------------------------
        
        self.P_time = BinPol(self.qubo_vs)   
        for t in range(self.N):
            self.P_time += (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for c in range(self.N))) ** 2
        self.P_time *= factor_rules
        
        #-P_city-------------------------------------------------------------------------------

        self.P_city = factor_rules *  BinPol.sum(
            (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for t in range(self.N))) ** 2 
            for  c in range(self.N))

        #-P_distance---------------------------------------------------------------------------
                        
        self.P_distance = factor_distance * BinPol.sum(Term(self.distance_table[c_0][c_1], (('x', t, c_0), ('x', (t + 1) % self.N, c_1)), self.qubo_vs)
                                                       for c_0 in range(self.N) for c_1 in range(self.N) if c_0 != c_1 for t in range(self.N))

        #-HQ-----------------------------------------------------------------------------------

        self.HQ = self.P_time + self.P_city + self.P_distance
        
        # calculate and set start temperature as standard deviation of QUBO energy
        recommended_temperature_start = round(0.1 * np.std([self.HQ.compute([randint(0, 1) for _ in range(self.N**2)]) for case in range(20)]), 3)
        self.Optimizer.set_calculated_solver_parameter(temperature_start=recommended_temperature_start)
        
        if not silent:
            print(f"""
    Polynomial created 
    Number of bits:               {self.HQ.N: 5d}
    Calculated start_temperature: {recommended_temperature_start: 9.3f}""")
    
    ##################################################################################################################
    # prep_result method (evaluates result of annealing defines content of tab Solve annealing)
    ##################################################################################################################
    def prep_result(self, solution_list:SolutionList, silent=False):

        configuration = solution_list.min_solution.configuration
        
        self.configuration = configuration # <-- best solution
        self.start_table = None
        self.route = None
              
        if not silent:
            e_HQ = self.HQ.compute(self.configuration)
            e_time = self.P_time.compute(self.configuration)
            e_city = self.P_city.compute(self.configuration)
            e_distance = self.P_distance.compute(self.configuration)
            
            print('Energy of QUBO / parts in best solution (P_time and P_city should be zero)')
            print("HQ         = %10.2F" % e_HQ)
            print("P_time     = %10.2F" % e_time)
            print("P_city     = %10.2F" % e_city)
            print("P_distance = %10.2F" % e_distance)
            print("")
            if e_time == 0 and e_city == 0:

                self.start_table = solution_list.min_solution.extract_bit_array('x')
                self.route = [np.argmax(self.start_table.data[t,:]) for t in range(self.N)]

                distances = [self.distance_table[self.route[t]][self.route[(t + 1) % self.N]] for t in range(self.N)]
                self.output_table = tabulate(zip(range(self.N),
                                                 self.route,
                                                 self.route[1:] + self.route[:1],
                                                 distances,
                                                 np.cumsum(distances)
                                                ),
                                             ['t', 'from', 'to', 'distance', 'total'], tablefmt="html")
                display(HTML(self.output_table))
            else:
                print("invalid solution, try again!")
           
    ##################################################################################################################
    # report method (defines content of Report tab)
    ##################################################################################################################
    def report(self, silent=False):
        
        if self.start_table is not None:
            self.start_table.draw(figsize=(6.0, 6.0))
            
    def report_title(self):
        return "Bit map report"
    
    ##################################################################################################################
    # report1 method (defines content of additional Report tab)
    ##################################################################################################################
    def report1(self, silent=False):
        print('This is the best found roundtrip:')
        if self.start_table is not None:
            display(HTML(self.output_table))
            
    def report1_title(self):
        return "Route report"
    
    ##################################################################################################################
    # draw method (defines content of visualization tab)
    ##################################################################################################################
    def draw(self):
        if self.start_table is not None and self.route is not None:
            self.graph = nx.Graph()

            nodes={}
            for i in range(self.N):
                nodes[i] = str(i);
                self.graph.add_node(nodes[i], pos=(self.cities[i][0], self.cities[i][1]))

            path = [[self.route[t], self.route[(t + 1) % self.N]] for t in range(self.N)]

            for i in range(self.N):
                for j in range(i+1, self.N):
                    if [i, j] in path or [j, i] in path: 
                        self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=10)
                    else:
                        self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=1)

            fig = plt.figure(num='Graph (created at %s)' % datetime.now(), figsize=(8.0, 8.0))

            pos=nx.get_node_attributes(self.graph,'pos')

            # nodes
            nx.draw_networkx_nodes(self.graph, pos, nodelist=self.graph.nodes, node_size=100, node_color="red")

            # edges        
            edges_bold = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == 10]
            nx.draw_networkx_edges(self.graph, pos, edgelist=edges_bold, width=3, edge_color="black")

            edges_thin = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == 1]
            nx.draw_networkx_edges(self.graph, pos, edgelist=edges_thin, width=1, alpha=0.1, edge_color="black")

            # labels
            nx.draw_networkx_labels(self.graph, pos, font_size=6, font_family='sans-serif')

            plt.axis('off')
            plt.show()
        
    ##################################################################################################################
    def pareto(self):
    ##################################################################################################################

        return {
            'HQ' : self.HQ.compute(self.configuration),
            'P_time': self.P_time.compute(self.configuration),
            'P_city': self.P_city.compute(self.configuration),
            'P_distance': self.P_distance.compute(self.configuration)
        }
        
optimizer = Optimizer(Optimizer_Demo_TSP_Model())

When you feel familiar with the steps in the GUI you can continue with the [next lesson](O_02_Optimizer_UI_functions.ipynb) about, how to control and program the complete model and optimization process.